# Part I: Scraping MTG Cards


For this project, I will be scraping the information available on the Card Kingdom website regarding Magic the Gathering cards.

In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import re
import time
import pandas as pd
import mtgutilities as mtgu
import json

In [3]:
with open('config.json', 'r') as config_file:
    contents = json.loads(config_file.read())

### Getting all of the expansion URLs

In [8]:
soup = mtgu.get_soup_from_website("https://www.cardkingdom.com/catalog/magic_the_gathering/by_az", options=True)
expansions = dict()
for a in soup.find_all('a', href=True):
        urls = dict()
        if 'mtg/' in a['href'] and 'art-series' not in a['href'] and 'graded-magic' \
        not in a['href'] and 'complete-sets' not in a['href'] and 'commander-deck' not in a['href'] \
        and 'card-kingdom-tokens' not in a['href'] and 'variants' not in a['href'] \
        and 'starter-kits' not in a['href'] and 'jpn' not in a['href']:
            
            urls["url"] = f"https://www.cardkingdom.com{a['href']}"
            expansions[a.string.replace(' ', '-').replace('---','-').replace(':','')\
                .replace('&','').replace('.','').replace("'", "").replace('--','-').replace('/','-').lower()] = urls

In [11]:
expansions = {'archenemy-nicol-bolas': {'url': 'https://www.cardkingdom.com/mtg/archenemy-nicol-bolas'},
 'avacyn-restored': {'url': 'https://www.cardkingdom.com/mtg/avacyn-restored'},
 'battle-for-zendikar': {'url': 'https://www.cardkingdom.com/mtg/battle-for-zendikar'},
 'battle-royale': {'url': 'https://www.cardkingdom.com/mtg/battle-royale'},
 'battlebond': {'url': 'https://www.cardkingdom.com/mtg/battlebond'},
 'beatdown': {'url': 'https://www.cardkingdom.com/mtg/beatdown'},
 'beta': {'url': 'https://www.cardkingdom.com/mtg/beta'},
 'betrayers-of-kamigawa': {'url': 'https://www.cardkingdom.com/mtg/betrayers-of-kamigawa'},
 'born-of-the-gods': {'url': 'https://www.cardkingdom.com/mtg/born-of-the-gods'},
 'challenger-decks-2018': {'url': 'https://www.cardkingdom.com/mtg/challenger-decks-2018'},
 'challenger-decks-2019': {'url': 'https://www.cardkingdom.com/mtg/challenger-decks-2019'},
 'challenger-decks-2020': {'url': 'https://www.cardkingdom.com/mtg/challenger-decks-2020'},
 'challenger-decks-2021': {'url': 'https://www.cardkingdom.com/mtg/challenger-decks-2021'},
 'challenger-decks-2022': {'url': 'https://www.cardkingdom.com/mtg/challenger-decks-2022'},
 'champions-of-kamigawa': {'url': 'https://www.cardkingdom.com/mtg/champions-of-kamigawa'},
 'chronicles': {'url': 'https://www.cardkingdom.com/mtg/chronicles'},
 'coldsnap': {'url': 'https://www.cardkingdom.com/mtg/coldsnap'},
 'coldsnap-theme-decks': {'url': 'https://www.cardkingdom.com/mtg/coldsnap-theme-decks'},
 'collectors-ed': {'url': 'https://www.cardkingdom.com/mtg/collectors-ed'},
 'collectors-ed-intl': {'url': 'https://www.cardkingdom.com/mtg/collectors-ed-intl'},
 'commander': {'url': 'https://www.cardkingdom.com/mtg/commander'},
 'commander-2013': {'url': 'https://www.cardkingdom.com/mtg/commander-2013'},
 'commander-2014': {'url': 'https://www.cardkingdom.com/mtg/commander-2014'},
 'commander-2015': {'url': 'https://www.cardkingdom.com/mtg/commander-2015'},
 'commander-2016': {'url': 'https://www.cardkingdom.com/mtg/commander-2016'},
 'commander-2017': {'url': 'https://www.cardkingdom.com/mtg/commander-2017'},
 'commander-2018': {'url': 'https://www.cardkingdom.com/mtg/commander-2018'},
 'commander-2019': {'url': 'https://www.cardkingdom.com/mtg/commander-2019'},
 'commander-2020': {'url': 'https://www.cardkingdom.com/mtg/commander-2020'},
 'commander-2021': {'url': 'https://www.cardkingdom.com/mtg/commander-2021'},
 'commander-anthology': {'url': 'https://www.cardkingdom.com/mtg/commander-anthology'},
 'commander-anthology-vol-ii': {'url': 'https://www.cardkingdom.com/mtg/commander-anthology-vol-ii'},
 'commander-collection-black': {'url': 'https://www.cardkingdom.com/mtg/commander-collection-black'},
 'commander-collection-green': {'url': 'https://www.cardkingdom.com/mtg/commander-collection-green'},
 'commander-legends': {'url': 'https://www.cardkingdom.com/mtg/commander-legends'},
 'commander-legends-battle-for-baldurs-gate': {'url': 'https://www.cardkingdom.com/mtg/commander-legends-battle-for-baldurs-gate'},
 'commanders-arsenal': {'url': 'https://www.cardkingdom.com/mtg/commanders-arsenal'},
 'conflux': {'url': 'https://www.cardkingdom.com/mtg/conflux'},
 'conspiracy': {'url': 'https://www.cardkingdom.com/mtg/conspiracy'},
 'conspiracy-take-the-crown': {'url': 'https://www.cardkingdom.com/mtg/conspiracy-take-the-crown'},
 'core-set-2019': {'url': 'https://www.cardkingdom.com/mtg/core-set-2019'},
 'core-set-2020': {'url': 'https://www.cardkingdom.com/mtg/core-set-2020'},
 'core-set-2021': {'url': 'https://www.cardkingdom.com/mtg/core-set-2021'},
 'dark-ascension': {'url': 'https://www.cardkingdom.com/mtg/dark-ascension'},
 'darksteel': {'url': 'https://www.cardkingdom.com/mtg/darksteel'},
 'deckmaster': {'url': 'https://www.cardkingdom.com/mtg/deckmaster'},
 'dissension': {'url': 'https://www.cardkingdom.com/mtg/dissension'},
 'dominaria': {'url': 'https://www.cardkingdom.com/mtg/dominaria'},
 'double-masters': {'url': 'https://www.cardkingdom.com/mtg/double-masters'},
 'double-masters-2022': {'url': 'https://www.cardkingdom.com/mtg/double-masters-2022'},
 'double-masters-box-toppers': {'url': 'https://www.cardkingdom.com/mtg/double-masters-box-toppers'},
 'dragons-maze': {'url': 'https://www.cardkingdom.com/mtg/dragons-maze'},
 'dragons-of-tarkir': {'url': 'https://www.cardkingdom.com/mtg/dragons-of-tarkir'},
 'duel-decks-ajani-vs-nicol-bolas': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-ajani-vs-nicol-bolas'},
 'duel-decks-anthology': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-anthology'},
 'duel-decks-blessed-vs-cursed': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-blessed-vs-cursed'},
 'duel-decks-divine-vs-demonic': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-divine-vs-demonic'},
 'duel-decks-elspeth-vs-kiora': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-elspeth-vs-kiora'},
 'duel-decks-elspeth-vs-tezzeret': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-elspeth-vs-tezzeret'},
 'duel-decks-elves-vs-goblins': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-elves-vs-goblins'},
 'duel-decks-elves-vs-inventors': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-elves-vs-inventors'},
 'duel-decks-garruk-vs-liliana': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-garruk-vs-liliana'},
 'duel-decks-heroes-vs-monsters': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-heroes-vs-monsters'},
 'duel-decks-izzet-vs-golgari': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-izzet-vs-golgari'},
 'duel-decks-jace-vs-chandra': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-jace-vs-chandra'},
 'duel-decks-jace-vs-vraska': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-jace-vs-vraska'},
 'duel-decks-knights-vs-dragons': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-knights-vs-dragons'},
 'duel-decks-merfolk-vs-goblins': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-merfolk-vs-goblins'},
 'duel-decks-mind-vs-might': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-mind-vs-might'},
 'duel-decks-nissa-vs-ob-nixilis': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-nissa-vs-ob-nixilis'},
 'duel-decks-phyrexia-vs-the-coalition': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-phyrexia-vs-the-coalition'},
 'duel-decks-sorin-vs-tibalt': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-sorin-vs-tibalt'},
 'duel-decks-speed-vs-cunning': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-speed-vs-cunning'},
 'duel-decks-venser-vs-koth': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-venser-vs-koth'},
 'duel-decks-zendikar-vs-eldrazi': {'url': 'https://www.cardkingdom.com/mtg/duel-decks-zendikar-vs-eldrazi'},
 'duels-of-the-planeswalkers': {'url': 'https://www.cardkingdom.com/mtg/duels-of-the-planeswalkers'},
 'eldritch-moon': {'url': 'https://www.cardkingdom.com/mtg/eldritch-moon'},
 'eternal-masters': {'url': 'https://www.cardkingdom.com/mtg/eternal-masters'},
 'eventide': {'url': 'https://www.cardkingdom.com/mtg/eventide'},
 'exodus': {'url': 'https://www.cardkingdom.com/mtg/exodus'},
 'explorers-of-ixalan': {'url': 'https://www.cardkingdom.com/mtg/explorers-of-ixalan'},
 'fallen-empires': {'url': 'https://www.cardkingdom.com/mtg/fallen-empires'},
 'fate-reforged': {'url': 'https://www.cardkingdom.com/mtg/fate-reforged'},
 'fifth-dawn': {'url': 'https://www.cardkingdom.com/mtg/fifth-dawn'},
 'from-the-vault-angels': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-angels'},
 'from-the-vault-annihilation': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-annihilation'},
 'from-the-vault-dragons': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-dragons'},
 'from-the-vault-exiled': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-exiled'},
 'from-the-vault-legends': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-legends'},
 'from-the-vault-lore': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-lore'},
 'from-the-vault-realms': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-realms'},
 'from-the-vault-relics': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-relics'},
 'from-the-vault-transform': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-transform'},
 'from-the-vault-twenty': {'url': 'https://www.cardkingdom.com/mtg/from-the-vault-twenty'},
 'future-sight': {'url': 'https://www.cardkingdom.com/mtg/future-sight'},
 'game-night': {'url': 'https://www.cardkingdom.com/mtg/game-night'},
 'game-night-2019': {'url': 'https://www.cardkingdom.com/mtg/game-night-2019'},
 'gatecrash': {'url': 'https://www.cardkingdom.com/mtg/gatecrash'},
 'gift-pack-2017': {'url': 'https://www.cardkingdom.com/mtg/gift-pack-2017'},
 'global-series-jiang-yanggu-mu-yanling': {'url': 'https://www.cardkingdom.com/mtg/global-series-jiang-yanggu-mu-yanling'},
 'guildpact': {'url': 'https://www.cardkingdom.com/mtg/guildpact'},
 'guilds-of-ravnica': {'url': 'https://www.cardkingdom.com/mtg/guilds-of-ravnica'},
 'guilds-of-ravnica-guild-kits': {'url': 'https://www.cardkingdom.com/mtg/guilds-of-ravnica-guild-kits'},
 'homelands': {'url': 'https://www.cardkingdom.com/mtg/homelands'},
 'hour-of-devastation': {'url': 'https://www.cardkingdom.com/mtg/hour-of-devastation'},
 'ice-age': {'url': 'https://www.cardkingdom.com/mtg/ice-age'},
 'iconic-masters': {'url': 'https://www.cardkingdom.com/mtg/iconic-masters'},
 'ikoria-lair-of-behemoths': {'url': 'https://www.cardkingdom.com/mtg/ikoria-lair-of-behemoths'},
 'innistrad': {'url': 'https://www.cardkingdom.com/mtg/innistrad'},
 'innistrad-crimson-vow': {'url': 'https://www.cardkingdom.com/mtg/innistrad-crimson-vow'},
 'innistrad-double-feature': {'url': 'https://www.cardkingdom.com/mtg/innistrad-double-feature'},
 'innistrad-midnight-hunt': {'url': 'https://www.cardkingdom.com/mtg/innistrad-midnight-hunt'},
 'invasion': {'url': 'https://www.cardkingdom.com/mtg/invasion'},
 'ixalan': {'url': 'https://www.cardkingdom.com/mtg/ixalan'},
 'journey-into-nyx': {'url': 'https://www.cardkingdom.com/mtg/journey-into-nyx'},
 'judgment': {'url': 'https://www.cardkingdom.com/mtg/judgment'},
 'jumpstart': {'url': 'https://www.cardkingdom.com/mtg/jumpstart'},
 'kaladesh': {'url': 'https://www.cardkingdom.com/mtg/kaladesh'},
 'kaldheim': {'url': 'https://www.cardkingdom.com/mtg/kaldheim'},
 'kamigawa-neon-dynasty': {'url': 'https://www.cardkingdom.com/mtg/kamigawa-neon-dynasty'},
 'khans-of-tarkir': {'url': 'https://www.cardkingdom.com/mtg/khans-of-tarkir'},
 'legends': {'url': 'https://www.cardkingdom.com/mtg/legends'},
 'legions': {'url': 'https://www.cardkingdom.com/mtg/legions'},
 'lorwyn': {'url': 'https://www.cardkingdom.com/mtg/lorwyn'},
 'magic-origins': {'url': 'https://www.cardkingdom.com/mtg/magic-origins'},
 'masterpiece-series-expeditions': {'url': 'https://www.cardkingdom.com/mtg/masterpiece-series-expeditions'},
 'masterpiece-series-inventions': {'url': 'https://www.cardkingdom.com/mtg/masterpiece-series-inventions'},
 'masterpiece-series-invocations': {'url': 'https://www.cardkingdom.com/mtg/masterpiece-series-invocations'},
 'masterpiece-series-mythic-edition': {'url': 'https://www.cardkingdom.com/mtg/masterpiece-series-mythic-edition'},
 'masters-25': {'url': 'https://www.cardkingdom.com/mtg/masters-25'},
 'mercadian-masques': {'url': 'https://www.cardkingdom.com/mtg/mercadian-masques'},
 'mirage': {'url': 'https://www.cardkingdom.com/mtg/mirage'},
 'mirrodin': {'url': 'https://www.cardkingdom.com/mtg/mirrodin'},
 'mirrodin-besieged': {'url': 'https://www.cardkingdom.com/mtg/mirrodin-besieged'},
 'modern-event-deck': {'url': 'https://www.cardkingdom.com/mtg/modern-event-deck'},
 'modern-horizons': {'url': 'https://www.cardkingdom.com/mtg/modern-horizons'},
 'modern-horizons-retro-frames': {'url': 'https://www.cardkingdom.com/mtg/modern-horizons-retro-frames'},
 'modern-horizons-2': {'url': 'https://www.cardkingdom.com/mtg/modern-horizons-2'},
 'modern-masters': {'url': 'https://www.cardkingdom.com/mtg/modern-masters'},
 'modern-masters-2015': {'url': 'https://www.cardkingdom.com/mtg/modern-masters-2015'},
 'modern-masters-2017': {'url': 'https://www.cardkingdom.com/mtg/modern-masters-2017'},
 'morningtide': {'url': 'https://www.cardkingdom.com/mtg/morningtide'},
 'mystery-booster-the-list': {'url': 'https://www.cardkingdom.com/mtg/mystery-booster-the-list'},
 'nemesis': {'url': 'https://www.cardkingdom.com/mtg/nemesis'},
 'new-phyrexia': {'url': 'https://www.cardkingdom.com/mtg/new-phyrexia'},
 'oath-of-the-gatewatch': {'url': 'https://www.cardkingdom.com/mtg/oath-of-the-gatewatch'},
 'odyssey': {'url': 'https://www.cardkingdom.com/mtg/odyssey'},
 'onslaught': {'url': 'https://www.cardkingdom.com/mtg/onslaught'},
 'pioneer-challenger-decks-2021': {'url': 'https://www.cardkingdom.com/mtg/pioneer-challenger-decks-2021'},
 'planar-chaos': {'url': 'https://www.cardkingdom.com/mtg/planar-chaos'},
 'planechase': {'url': 'https://www.cardkingdom.com/mtg/planechase'},
 'planechase-2012': {'url': 'https://www.cardkingdom.com/mtg/planechase-2012'},
 'planechase-anthology': {'url': 'https://www.cardkingdom.com/mtg/planechase-anthology'},
 'planeshift': {'url': 'https://www.cardkingdom.com/mtg/planeshift'},
 'portal': {'url': 'https://www.cardkingdom.com/mtg/portal'},
 'portal-3k': {'url': 'https://www.cardkingdom.com/mtg/portal-3k'},
 'portal-ii': {'url': 'https://www.cardkingdom.com/mtg/portal-ii'},
 'premium-deck-series-fire-lightning': {'url': 'https://www.cardkingdom.com/mtg/premium-deck-series-fire-lightning'},
 'premium-deck-series-graveborn': {'url': 'https://www.cardkingdom.com/mtg/premium-deck-series-graveborn'},
 'premium-deck-series-slivers': {'url': 'https://www.cardkingdom.com/mtg/premium-deck-series-slivers'},
 'promo-pack': {'url': 'https://www.cardkingdom.com/mtg/promo-pack'},
 'promotional': {'url': 'https://www.cardkingdom.com/mtg/promotional'},
 'prophecy': {'url': 'https://www.cardkingdom.com/mtg/prophecy'},
 'ravnica': {'url': 'https://www.cardkingdom.com/mtg/ravnica'},
 'ravnica-allegiance': {'url': 'https://www.cardkingdom.com/mtg/ravnica-allegiance'},
 'ravnica-allegiance-guild-kits': {'url': 'https://www.cardkingdom.com/mtg/ravnica-allegiance-guild-kits'},
 'return-to-ravnica': {'url': 'https://www.cardkingdom.com/mtg/return-to-ravnica'},
 'rise-of-the-eldrazi': {'url': 'https://www.cardkingdom.com/mtg/rise-of-the-eldrazi'},
 'rivals-of-ixalan': {'url': 'https://www.cardkingdom.com/mtg/rivals-of-ixalan'},
 'saviors-of-kamigawa': {'url': 'https://www.cardkingdom.com/mtg/saviors-of-kamigawa'},
 'scars-of-mirrodin': {'url': 'https://www.cardkingdom.com/mtg/scars-of-mirrodin'},
 'scourge': {'url': 'https://www.cardkingdom.com/mtg/scourge'},
 'secret-lair': {'url': 'https://www.cardkingdom.com/mtg/secret-lair'},
 'shadowmoor': {'url': 'https://www.cardkingdom.com/mtg/shadowmoor'},
 'shadows-over-innistrad': {'url': 'https://www.cardkingdom.com/mtg/shadows-over-innistrad'},
 'shards-of-alara': {'url': 'https://www.cardkingdom.com/mtg/shards-of-alara'},
 'signature-spellbook-chandra': {'url': 'https://www.cardkingdom.com/mtg/signature-spellbook-chandra'},
 'signature-spellbook-gideon': {'url': 'https://www.cardkingdom.com/mtg/signature-spellbook-gideon'},
 'signature-spellbook-jace': {'url': 'https://www.cardkingdom.com/mtg/signature-spellbook-jace'},
 'starter-1999': {'url': 'https://www.cardkingdom.com/mtg/starter-1999'},
 'starter-2000': {'url': 'https://www.cardkingdom.com/mtg/starter-2000'},
 'streets-of-new-capenna': {'url': 'https://www.cardkingdom.com/mtg/streets-of-new-capenna'},
 'strixhaven-mystical-archive': {'url': 'https://www.cardkingdom.com/mtg/strixhaven-mystical-archive'},
 'strixhaven-school-of-mages': {'url': 'https://www.cardkingdom.com/mtg/strixhaven-school-of-mages'},
 'stronghold': {'url': 'https://www.cardkingdom.com/mtg/stronghold'},
 'tempest': {'url': 'https://www.cardkingdom.com/mtg/tempest'},
 'the-dark': {'url': 'https://www.cardkingdom.com/mtg/the-dark'},
 'theros': {'url': 'https://www.cardkingdom.com/mtg/theros'},
 'theros-beyond-death': {'url': 'https://www.cardkingdom.com/mtg/theros-beyond-death'},
 'throne-of-eldraine': {'url': 'https://www.cardkingdom.com/mtg/throne-of-eldraine'},
 'time-spiral': {'url': 'https://www.cardkingdom.com/mtg/time-spiral'},
 'time-spiral-remastered': {'url': 'https://www.cardkingdom.com/mtg/time-spiral-remastered'},
 'timeshifted': {'url': 'https://www.cardkingdom.com/mtg/timeshifted'},
 'torment': {'url': 'https://www.cardkingdom.com/mtg/torment'},
 'ultimate-box-topper': {'url': 'https://www.cardkingdom.com/mtg/ultimate-box-topper'},
 'ultimate-masters': {'url': 'https://www.cardkingdom.com/mtg/ultimate-masters'},
 'unglued': {'url': 'https://www.cardkingdom.com/mtg/unglued'},
 'unhinged': {'url': 'https://www.cardkingdom.com/mtg/unhinged'},
 'unlimited': {'url': 'https://www.cardkingdom.com/mtg/unlimited'},
 'unsanctioned': {'url': 'https://www.cardkingdom.com/mtg/unsanctioned'},
 'unstable': {'url': 'https://www.cardkingdom.com/mtg/unstable'},
 'urzas-destiny': {'url': 'https://www.cardkingdom.com/mtg/urzas-destiny'},
 'urzas-legacy': {'url': 'https://www.cardkingdom.com/mtg/urzas-legacy'},
 'urzas-saga': {'url': 'https://www.cardkingdom.com/mtg/urzas-saga'},
 'vanguard': {'url': 'https://www.cardkingdom.com/mtg/vanguard'},
 'visions': {'url': 'https://www.cardkingdom.com/mtg/visions'},
 'war-of-the-spark': {'url': 'https://www.cardkingdom.com/mtg/war-of-the-spark'},
 'weatherlight': {'url': 'https://www.cardkingdom.com/mtg/weatherlight'},
 'world-championships': {'url': 'https://www.cardkingdom.com/mtg/world-championships'},
 'worldwake': {'url': 'https://www.cardkingdom.com/mtg/worldwake'},
 'zendikar': {'url': 'https://www.cardkingdom.com/mtg/zendikar'},
 'zendikar-rising': {'url': 'https://www.cardkingdom.com/mtg/zendikar-rising'},
 'zendikar-rising-expeditions': {'url': 'https://www.cardkingdom.com/mtg/zendikar-rising-expeditions'}}

In [14]:
expansions_df = pd.DataFrame()
for expansion_key, expansion_values in expansions.items():
    # Obtaining number of pages to scrape per expansion
    soup = mtgu.get_soup_from_website(expansion_values.get("url"), options=True)
    max_pages = mtgu.get_max_pages(soup)

    # Scraping page by page
    driver_pages = webdriver.Chrome(contents["chrome-driver-location"])
    
    card_names, card_costs, card_converted_costs, card_type, \
        card_urls, card_rarity, card_pt, card_text, card_price = list(), list(), list(), list(), list(), list(), list(), list(), list()

    for i in range(1,max_pages+1):
            driver_pages.get(f"{expansion_values.get('url')}/?page={str(i)}")
            soup_pages = bs(driver_pages.page_source, 'html.parser')
            
            # Getting card URLs
            card_urls += mtgu.get_card_urls(soup_pages, expansion_key)
            # Getting card names
            card_names += mtgu.get_card_names(soup_pages, expansion_key)
            # Getting mana costs
            card_costs += mtgu.get_mana_cost(soup_pages)
            # Getting converted mana cost
            card_converted_costs = mtgu.get_converted_mana_cost(card_costs)
            # Getting card type
            card_type += mtgu.get_card_type(soup_pages)
            # Getting card rarity
            card_rarity += mtgu.get_card_rarity(soup_pages)
            # Getting card P/T
            card_pt += mtgu.get_card_pt(soup_pages)
            # Getting card text
            card_text += mtgu.get_card_text(soup_pages)
            # Getting card price
            card_price += mtgu.get_card_price(soup_pages)
            time.sleep(3)


    df = pd.DataFrame(
    {'expansion_id': expansion_key,
     'url': card_urls,
     'name': card_names,
     'cost': card_costs,
     'converted_cost': card_converted_costs,
     'type': card_type,
     'rarity': card_rarity,
     'pt': card_pt,
     'text': card_text,
     'price': card_price
    })
    
    expansions_df.to_csv(f"{contents['external-drive-location']}/cards/cards.csv", mode='a', index=False,header=False)          

KeyboardInterrupt: 